In [1]:
import seaborn as sns
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [26]:
df = pd.read_csv('../DataScience/DATA/AMES_Final_DF.csv')

In [27]:
df.head()

,Lot Frontage,Lot Area,Overall Qual,Overall Cond,Year Built,Year Remod/Add,Mas Vnr Area,BsmtFin SF 1,BsmtFin SF 2,Bsmt Unf SF,...,Sale Type_ConLw,Sale Type_New,Sale Type_Oth,Sale Type_VWD,Sale Type_WD,Sale Condition_AdjLand,Sale Condition_Alloca,Sale Condition_Family,Sale Condition_Normal,Sale Condition_Partial
0,141.0,31770,6,5,1960,1960,112.0,639.0,0.0,441.0,...,0,0,0,0,1,0,0,0,1,0
1,80.0,11622,5,6,1961,1961,0.0,468.0,144.0,270.0,...,0,0,0,0,1,0,0,0,1,0
2,81.0,14267,6,6,1958,1958,108.0,923.0,0.0,406.0,...,0,0,0,0,1,0,0,0,1,0
3,93.0,11160,7,5,1968,1968,0.0,1065.0,0.0,1045.0,...,0,0,0,0,1,0,0,0,1,0
4,74.0,13830,5,5,1997,1998,0.0,791.0,0.0,137.0,...,0,0,0,0,1,0,0,0,1,0


In [28]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2925 entries, 0 to 2924
Columns: 274 entries, Lot Frontage to Sale Condition_Partial
dtypes: float64(11), int64(263)
memory usage: 6.1 MB


In [29]:
X = df.drop('SalePrice', axis=1)

In [30]:
y = df['SalePrice']

In [31]:
from sklearn.model_selection import train_test_split

In [32]:
# Will use Kfold Cross Validation so test size is only final hold out test set. 

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=101)

In [33]:
# Will scale features

from sklearn.preprocessing import StandardScaler

In [34]:
scaler = StandardScaler()

In [59]:
#Fits and transforms on X_train and X_test

scaled_X_train = scaler.fit_transform(X_train)
scaled_X_test = scaler.transform(X_test)

In [60]:
#Will use elastic net model 

from sklearn.linear_model import ElasticNet

In [61]:
#Set max interations for the gradient decent 

basic_elastic_model = ElasticNet(max_iter=1000000)

In [62]:
# Create grid search parameters that will cylce through serveral weights utilzing lasso and ridge regression
param_grid = {'alpha':[0.1,1,5,10,100],'l1_ratio':[.1,.7,.99,1]}

In [63]:
from sklearn.model_selection import GridSearchCV

In [64]:
# Setup grid model to use our elastic net regression, 
#the grid parameters, the soreing metric that will be used to 
#determine the best fit and the nuvmer fo cross validations folds.

grid_model = GridSearchCV(base_elastic_model, 
                          param_grid=param_grid,
                         scoring='neg_mean_squared_error',
                         cv=5, verbose=1)

In [65]:
grid_model.fit(scaled_X_train,y_train)

Fitting 5 folds for each of 20 candidates, totalling 100 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
/Users/adolfo/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:529: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 354244712120.8144, tolerance: 1355206692.5276787
  model = cd_fast.enet_coordinate_descent(
/Users/adolfo/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:529: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 363490863623.8193, tolerance: 1307913805.6588454
  model = cd_fast.enet_coordinate_descent(
/Users/adolfo/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:529: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 408539308800.7921, tolerance: 1415056940.006106
  model = cd_fast.enet_coordin

GridSearchCV(cv=5, estimator=ElasticNet(),
             param_grid={'alpha': [0.1, 1, 5, 10, 100],
                         'l1_ratio': [0.1, 0.7, 0.99, 1]},
             scoring='neg_mean_squared_error', verbose=1)

In [66]:
# Display the best parameters that the model produced

grid_model.best_params_

{'alpha': 100, 'l1_ratio': 1}

In [67]:
# Model tells us that lasso regression is the best model, 
# possibly due to the number of hyperperameters.

In [69]:
# Usethe 10% hold out to determine the model performance

y_pred = grid_model.predict(scaled_X_test)

In [70]:
from sklearn.metrics import mean_absolute_error,mean_squared_error

In [71]:
#Is not sensitve to extreme outliers,compared to MSE
# Closer to zero, the better
mean_absolute_error(y_test,y_pred)

14195.354900562164

In [80]:
# Our two measurments on the best fit model (MAE,RMSE)
#RMSE is in units of the data

In [75]:
np.sqrt(mean_squared_error(y_test,y_pred))

20558.508566893164

In [78]:
# In order to determine if the MAE/RMSE is "good" 
# we need to find the mean and compare

In [79]:
np.mean(df['SalePrice'])

180815.53743589742

In [ ]:
# We may want to back and remove hyperperameters to attempt to increase accruacy